In [2]:
import pandas as pd
import numpy as np
import os
import sys
import json
import re
import time
import pathlib
from pathlib import Path
import janitor
from janitor import clean_names
import datetime

In [15]:
reporting_metrics_df = pd.read_excel(
    data_path, sheet_name="Information Source Characterist", skiprows=1
)

In [ ]:
# set current_dir to the directory of the notebook
current_dir = Path().resolve()  # This will get the current working directory
# load company total electricity use data
reporting_metrics_df = pd.read_excel(
    data_path, sheet_name="Information Source Characterist", skiprows=1
)
print(f"\nInitial company total records: {len(reporting_metrics_df)}")

data_path = current_dir.parent / "data" / "modules.xlsx"
columns_to_melt = [
    "Total company electricity use reporting?",
    "Total data center fleet electricity use reporting?",
    "Individual data center electricity use reporting?",
    "Data center fuel use reporting?",
    "PUE reporting?",
    "Total company water use reporting?",
    "Total data center fleet water  use reporting?",
    "Individual data center water use reporting?",
    "WUE reporting?",
    "Total company electric power sources reporting?",
    "Data center fleet electric power sources reporting?",
    "Individual data center electric power sources reporting?",
    "Third-party standards utilization?",
    "Total company Scope 1 GHG reporting?",
    "Data center fleet Scope 1 GHG reporting?",
    "Individual data center Scope 1 GHG reporting?",
    "Total company Scope 2 GHG reporting?",
    "Data center fleet Scope 2 GHG reporting?",
    "Individual data center Scope 2 GHG reporting?",
    "Total company Scope 3 GHG reporting?",
    "Data center fleet Scope 3 GHG reporting?",
    "Individual data center Scope 3 GHG reporting?",
]
columns_to_keep = [
    "Company",
    "Title",
    "Information source type",
    "File name",
    "Date released ",
    "Data year convention",
    "Fiscal year start",
    "Fiscal year end",
    "Geographical scope",
    "If partial, which locations are included?",
    "Source citation",
]
# Melt the columns into rows
reporting_metrics_df = pd.melt(
    reporting_metrics_df,
    id_vars=columns_to_keep,
    value_vars=columns_to_melt,
    var_name="metric",
    value_name="metric_value",
)
# extract the year from the file name value
reporting_metrics_df["report_year"] = reporting_metrics_df["File name"].str.extract(
    r"(?<!\d)(2\d{3})(?!\d)"
)
# extract the year from the title value if the report_year is NaN
title_years = reporting_metrics_df["Title"].str.extract(r"(?<!\d)(2\d{3})(?!\d)")[0]
# extract and flatten to Series
reporting_metrics_df["report_year"] = reporting_metrics_df["report_year"].fillna(
    title_years
)
# remove trailing spaces from the company name
reporting_metrics_df["Company"] = reporting_metrics_df["Company"].str.strip()

reporting_metrics_df["report_year"] = pd.to_numeric(
    reporting_metrics_df["report_year"], errors="coerce"
)

reporting_metrics_df.dropna(subset=["report_year"], inplace=True)
reporting_metrics_df["report_year"] = reporting_metrics_df["report_year"].astype(int)

# get the latest report year for each company
most_recent_years = (
    reporting_metrics_df.groupby("Company")["report_year"].max().reset_index()
)

# merge to get the latest metrics and values
recent_metrics_df = reporting_metrics_df.merge(
    most_recent_years, on=["Company", "report_year"]
)

export_path = current_dir.parent / "data" / "reporting_metrics_df.csv"
reporting_metrics_df.to_csv(export_path, index=False)
export_path = current_dir.parent / "data" / "recent_metrics_df.csv"
recent_metrics_df.to_csv(export_path, index=False)

reporting_metrics_df[reporting_metrics_df["Company"] == "Adobe"].head()



Initial company total records: 259


,Company,Title,Information source type,File name,Date released,Data year convention,Fiscal year start,Fiscal year end,Geographical scope,"If partial, which locations are included?",Source citation,metric,metric_value,report_year
193,Adobe,Adobe CDP Climate Change Questionnaire 2023,Annual Report,Adobe_Environmental_Report_CDP,7/27/ 2023,Fiscal,2022-12-03 00:00:00,2023-12-02 00:00:00,Full,NaN,Adobe. Adobe CDP Climate Change Questionnaire ...,Total company electricity use reporting?,N,2023
194,Adobe,Adobe - Climate Change 2022,Annual Report,Adobe_Environmental_Report_CDP_2022,NaN,Fiscal,2020-12-01 00:00:00,2021-11-30 00:00:00,Full,NaN,Adobe. Adobe - Climate Change 2022. 2023. http...,Total company electricity use reporting?,N,2022
195,Adobe,Adobe CDP Climate Change Questionnaire 2021,Annual Report,Adobe_Environmental_Report_CDP_2021,8/24/2020,Fiscal,2019-11-30 00:00:00,2020-11-29 00:00:00,Full,NaN,Adobe. Adobe CDP Climate Change Questionnaire ...,Total company electricity use reporting?,N,2021
196,Adobe,Adobe CDP Climate Change Questionnaire 2020,Annual Report,Adobe_Environmental_Report_CDP_2020,7/30/2021,Fiscal,2018-12-01 00:00:00,2019-11-30 00:00:00,Full,NaN,Adobe. Adobe CDP Climate Change Questionnaire ...,Total company electricity use reporting?,N,2020
197,Adobe,"Adobe, Inc. - Climate Change 2019",Annual Report,Adobe_Environmental_Report_CDP_2019,NaN,Fiscal,2018-12-01 00:00:00,2019-11-30 00:00:00,Full,NaN,"Adobe. Adobe, Inc. - Climate Change 2019. 2020...",Total company electricity use reporting?,N,2019


In [44]:
    import pandas as pd
    from pathlib import Path
    import janitor
    from janitor import clean_names
    import datetime

    current_dir = Path().resolve()
    data_path = current_dir.parent / "data" / "modules.xlsx"
    # load company total electricity use data
    company_total_ec_df = pd.read_excel(
        data_path, sheet_name="Company Total Electricity Use", skiprows=1
    )
    print(f"\nInitial company total records: {len(company_total_ec_df)}")

    company_total_ec_df = company_total_ec_df.clean_names()

    # First rename, then clean
    company_total_ec_df.rename(
        columns={
            "company": "company_name",
            "reported_total_company_electricity_use_kwh_": "electricity_usage_kwh",
        },
        inplace=True,
    )

    # Clean company names
    company_total_ec_df["company_name"] = company_total_ec_df[
        "company_name"
    ].str.strip()

    # company_total_ec_df = company_total_ec_df[
    #     ["company_name", "reported_data_year", "electricity_usage_kwh"]
    # ]
    # print("\nCompany total data sample:")
    # print(company_total_ec_df.head())
    # print(
    #     f"Null values in company total electricity: {company_total_ec_df['electricity_usage_kwh'].isna().sum()}"
    # )

    # # load data center electricity use data
    # dc_ec_df = pd.read_excel(
    #     data_path, sheet_name="Data Center Electricity Use ", skiprows=1
    # )
    # print(f"\nInitial data center records: {len(dc_ec_df)}")

    # dc_ec_df = dc_ec_df.clean_names()

    # # First rename, then clean
    # dc_ec_df.rename(
    #     columns={
    #         "company": "company_name",
    #         "reported_data_center_electricity_use_kwh_": "electricity_usage_kwh",
    #     },
    #     inplace=True,
    # )

    # # Clean company names
    # dc_ec_df["company_name"] = dc_ec_df["company_name"].str.strip()

    # dc_ec_df = dc_ec_df[["company_name", "reported_data_year", "electricity_usage_kwh"]]

    # # Clean electricity usage values - preserve numbers that are already clean
    # dc_ec_df["electricity_usage_kwh"] = dc_ec_df["electricity_usage_kwh"].apply(
    #     lambda x: str(x).replace("<", "").replace(",", "") if isinstance(x, str) else x
    # )
    # dc_ec_df["electricity_usage_kwh"] = pd.to_numeric(
    #     dc_ec_df["electricity_usage_kwh"], errors="coerce"
    # )

    # print("\nData center data sample:")
    # print(dc_ec_df.head())
    # print(
    #     f"Null values in data center electricity: {dc_ec_df['electricity_usage_kwh'].isna().sum()}"
    # )

    # # Group by company and year for data centers
    # dc_ec_df = (
    #     dc_ec_df.groupby(["company_name", "reported_data_year"])[
    #         "electricity_usage_kwh"
    #     ]
    #     .sum()
    #     .reset_index()
    # )

    # # add a reporting_scope column to each DataFrame
    # company_total_ec_df["reporting_scope"] = "Company Wide Electricity Use"
    # dc_ec_df["reporting_scope"] = "Data Center Electricity Use"

    # # combine all the dfs into one
    # energy_use_df = pd.concat([company_total_ec_df, dc_ec_df], axis=0)
    # energy_use_df["reported_data_year"] = energy_use_df["reported_data_year"].astype(
    #     int
    # )

    # print("\nFinal dataset:")
    # print(f"Total records: {len(energy_use_df)}")
    # print(f"Unique companies: {len(energy_use_df['company_name'].unique())}")
    # print(
    #     f"Years range: {energy_use_df['reported_data_year'].min()} - {energy_use_df['reported_data_year'].max()}"
    # )
    # print(
    #     f"Null values in final electricity usage: {energy_use_df['electricity_usage_kwh'].isna().sum()}"
    # )
    # print("\nSample of final data:")
    # print(energy_use_df.head())
    # print(energy_use_df.columns)


Initial company total records: 199


In [45]:
company_total_ec_df["company_name"].unique()
company_total_ec_df[company_total_ec_df["company_name"] == "Baidu"]

# check electricity_usage_kwh type
# company_total_ec_df["electricity_usage_kwh"].astype(int)

# convert company_name to string
# company_total_ec_df['company_name'] = company_total_ec_df['company_name'].astype(str)


,input_by,status,checked_by,date_checked,company_name,reported_data_year,total_company_electricity_use_reporting_scope,reported_total_company_electricity_use_quantity,reported_units,electricity_usage_kwh,...,external_notes,internal_notes,source_column_s_#1,corresponding_source,associtated_page_or_table_number,source_column_s_#2,corresponding_source_1,associtated_page_or_table_number_1,unnamed_23,check
98,Kaia Moehlis,NaN,Ella Hartmanis,1/1/2025,Baidu,2023,Total operations,"1,020,507.6",MWh,1020507600,...,NaN,NaN,"CE4, CE7","Baidu. Baidu 2023 Environmental, Social and Go...",Page 27,NaN,NaN,NaN,NaN,0
99,Kaia Moehlis,NaN,Ella Hartmanis,1/1/2025,Baidu,2022,Total operations,"902,076",MWh,902076000,...,NaN,NaN,"CE4, CE7","Baidu. Baidu 2023 Environmental, Social and Go...",Page 27,NaN,NaN,NaN,NaN,0
100,Kaia Moehlis,NaN,Ella Hartmanis,1/1/2025,Baidu,2021,Total operations,"673,829.3",MWh,673829300,...,NaN,NaN,"CE4, CE7","Baidu. Baidu 2023 Environmental, Social and Go...",Page 27,NaN,NaN,NaN,NaN,0
101,Kaia Moehlis,NaN,Ella Hartmanis,1/1/2025,Baidu,2020,Total operations,"529,137.4",MWh,529137400,...,NaN,NaN,"CE4, CE7","Baidu. Baidu 2022 Environmental, Social and Go...",Page 32,NaN,NaN,NaN,NaN,0
102,Kaia Moehlis,NaN,Ella Hartmanis,1/1/2025,Baidu,2019,Total operations,"490,628,065.2",kWh,490628065.2,...,NaN,NaN,"CE4, CE7","Baidu. Baidu 2019 Environmental, Social and Go...",Page 36,NaN,NaN,NaN,NaN,0
